##  `Oracle for Shor's algorithm`

Shor's Algorithm solves the discrete logarithmic problem as follows: Given $a,N\in\Z_+$ such that $0<a<N$ then there exist $r\in \mathbb{Z}_+$ such that

   $$ a^r\equiv1 \mod N$$
We need to find the smallest $r$ that satisfies the above modular equation.
In this notebook we will build the oracle for the quantum component of Shor's Algorithm, in other words we need to constract an operator that does the following:


We take two inputs $a,N\in\mathbb{N}$ such that $a<N$ and output the following:

$$
U\ket{x}_{1}\ket{y}_{n} =
\begin{array}{ll}
    \ket{x}_1\ket{ay \bmod N} & \text{if } x=1 \text{ and } 0 \leq x < N \\
    \ket{x}_1\ket{y}          & \text{otherwise}
\end{array}
$$
We will try to implement this circuit following Figure 7 of Stéphane Beauregard, "Circuit for Shor's algorithm using 2n+3 qubits". Our $n$ is the biggest closesest integer to $2\log(N)$ and we are going to use 3 ancillas as well.

In [10]:
from qiskit import QuantumCircuit, QuantumRegister, AncillaRegister
from qiskit.circuit import Parameter
from qiskit.quantum_info import Statevector, Operator
from qiskit.circuit.library import QFT
from qiskit.visualization import matplotlib as _  # Ensures Qiskit sets up its matplotlib styles
import numpy as np

In [11]:
def precompute_modular_angles(N, a):

    n = int(np.ceil(np.log2(N)))
    
    # Rotation angles for modular multiplication by a:
    # a_pow_angles[i][j] ≈ (a * 2^i mod N) * π / 2^j
    a_pow_angles = [
        [(a * (1 << i) % N) * np.pi / (1 << j) for j in range(n + 1)]
        for i in range(n + 1)
    ]
    
    # Rotation angles for modular reduction by N:
    # N_angles[j] ≈ N * π / 2^j
    N_angles = [(N * np.pi) / (1 << j) for j in range(n + 1)]
    
    # Modular inverse of a modulo N
    a_inv = pow(a, -1, N)
    
    # Rotation angles for modular multiplication by a^{-1}:
    # a_inv_pow_angles[i][j] ≈ (a^{-1} * 2^i mod N) * π / 2^j
    a_inv_pow_angles = [
        [(a_inv * (1 << i) % N) * np.pi / (1 << j) for j in range(n + 1)]
        for i in range(n + 1)
    ]
    
    return a_pow_angles, N_angles, a_inv, a_inv_pow_angles

This function computes classical modular multiplication using phase rotation, we need this to simulate arithmetic operations in the Fourier basis.

Here’s what the function does:

- `n`: Computes the number of bits needed to represent the modulus `N` (i.e., ⌈log₂(N)⌉).
- `a_pow_angles[i][j]`: A 2D list of angles used to build phase rotations for modular multiplication by `a`. Each entry corresponds to:
  
  
  $$\theta_{i,j} = \frac{a \cdot 2^i \bmod N}{2^j} \cdot \pi$$
  

  This gives the contribution of bit position `i` to a rotation of precision `2^{-j}`.

- `N_angles[j]`: A list of angles used for conditional modular reduction. Each angle corresponds to:

  
  $$\theta_j = \frac{N \cdot \pi}{2^j}$$
  

- `a_inv`: The modular inverse of `a mod N`, used to reverse modular multiplication during uncomputation.
- `a_inv_pow_angles[i][j]`: Analogous to `a_pow_angles`, but for modular multiplication by `a⁻¹ mod N`.

All angles are scaled by powers of two and multiplied by π to match the required format for controlled phase rotations in quantum circuits.

In [7]:
# We now define our Quantum Register setup 

# Control qubit: determines if modular multiplication is applied
control_qubit = QuantumRegister(1, name="ctrl")

# Input register: holds |y⟩, the value to be multiplied by 'a' mod N
input_reg = QuantumRegister(n, name="input")

# Output register: holds the result of the desired modular multiplication
output_reg = QuantumRegister(n, name="output")

# Ancilla qubits: used for temporary computation (we will need 3 ancilla qubits)
ancilla_reg = QuantumRegister(3, name="anc")

# Construct the circit that combines everything together
mod_mult_circ = QuantumCircuit(control_qubit, input_reg, output_reg, ancilla_reg, name="mod_mult")

In this next block uses the Quantum Fourier Transform (QFT) to perform a modular subtraction of `N` from the combined quantum state stored across the `output`, `input`, and one ancilla qubit. The goal is to check whether the value $ y = \texttt{[output,input]} $ is less than `N`.

1. Apply the QFT to the combined register (`output`, `input`, and `ancilla[0]`) to work in the Fourier basis.
2. Subtract `N` by applying carefully calibrated phase rotations to simulate \( y - N \).
3. Perform the inverse QFT to return to the computational basis.
4. Use a CNOT to mark `ancilla[2]` if an underflow occurred (i.e., if \( y < N \)).
5. Add `N` back (using the same phase rotation trick) to restore the original value of `y`.

At the end, the quantum state is untouched **except for `ancilla[2]`**, which now flags whether $y < N $. This flag is crucial for conditional logic used later in the modular multiplication circuit.

In [12]:
# Create list of qubits representing the full y register:
# Concatenates: output_reg[0 to n-1], input_reg[0 to n-1], ancilla_reg[0]
yreg = list(output_reg) + list(input_reg) + [ancilla_reg[0]]

# Apply QFT to prepare for subtraction
mod_mult_circ.compose(QFT(len(yreg)), qubits=yreg, inplace=True)

# Subtract N by applying phase rotations (acts like y - N in Fourier basis)
mod_mult_circ.p(-N * np.pi, ancilla_reg[0])
for k in range(1, n + 1):
    mod_mult_circ.p(-N * np.pi / (1 << k), output_reg[n - k])
    mod_mult_circ.p(-N * np.pi / (1 << (n + k)), input_reg[n - k])

# Inverse QFT to return to computational basis
mod_mult_circ.compose(QFT(len(yreg)).inverse(), qubits=yreg, inplace=True)

# Mark ancilla[2] = 1 iff underflow occurred (i.e., y < N)
mod_mult_circ.cx(ancilla_reg[0], ancilla_reg[2])

# Add N back to restore the state (but ancilla[2] remembers underflow)
mod_mult_circ.compose(QFT(len(yreg)), qubits=yreg, inplace=True)
mod_mult_circ.p(N * np.pi, ancilla_reg[0])
for k in range(1, n + 1):
    mod_mult_circ.p(N * np.pi / (1 << k), output_reg[n - k])
    mod_mult_circ.p(N * np.pi / (1 << (n + k)), input_reg[n - k])
mod_mult_circ.compose(QFT(len(yreg)).inverse(), qubits=yreg, inplace=True)